# mysql基础

### 体系结构
* 连接池->管理服务工具->SQL接口组件->查询分析器(SQL解析器)->优化器组件(SQL优化器)->缓存组件->存储引擎->物理文件，如下图所示：
<img src="../images/mysql/mysql架构.png" width="400px">
* 处理查询流程图：
<img src='../images/mysql/mysql处理查询流程.png' width='400px'>

### 日志文件：
* 错误日志：
    * 记录mysql启动、运行、关闭过程的错误信息
    * ```SHOW VARIABLES like 'log_error'```，显示错误日志路径
* 慢查询日志：
    * 记录查询时间超过```long_query_time```阈值或者没使用索引的查询语句
    * ```SHOW VARIABLES like 'log_slow_%'```：查看记录慢查询日志是否开启
    * ```SHOW VARIABLES like 'log_queries_not_using_indexes'```：该变量用于设置是否记录没使用索引的查询语句
    * mysql5.1可以将慢查询日志记录在mysql.slow_log表，通过log_output参照指定日志输出格式是表还是文件
    * mysqldumpslow工具处理慢查询日志，解析工具pt-query-digest
* 查询日志：
    * 以主机名命名的文件：host.log
    *  mysql5.1可以将查询日志记录在general_log表
* 二进制日志binlog：
    * 记录对数据库更改的所有操作，可用于时间点恢复和主从数据库复制
    * 记录具体的操作，包括所有存储引擎，并且只记录提交后的事务
    * ```log-bin=[name]```：启动二进制日志
    * ```binlog_cache_size```：指定二进制日志写缓存大小，当处理事务时，会将操作记录写入缓存，当缓存超过该值时，则写入到临时文件，最后事务提交后才会写入到binlog
    * ```sync_binlog```：指定每写多少次缓存就将数据同步到磁盘文件，这个值会与事务未提交产生冲突，因为当这个值设置为1时，即使事务未提交，也会将记录写入到binlog文件中
    * ```binlog_format```：指定binlog的记录格式，可以是基于语句，基于行数据或者混合模式
    * ```max_binlog_size```：指定单个二进制文件的最大值
    * ```binlog-do-db```：指定要记录二进制日志的数据库
    * ```binlog-ignore-db```：指定要忽略记录二进制日志的数据库
    * ```log-slave-update```：指定操作执行master的二进制日志时是否也记录到slave的二进制日志中，用于搭建master->slave->slave架构
    * mysqlbinlog工具处理二进制日志文件
    
### 视图：本质是一张虚表
* 创建视图时加上WITH CHECK OPTION，则在插入数据到视图时会做数据一致性检查
* 物化视图更新模式：
    * ON DEMAND：需要时才更新物化视图
    * ON COMMIT：每次完成DML操作提交后进行更新
* 创建视图：
    ```sql
    CREATE VIEW view_name [(column_list)]
        AS select_statement
        [WITH [CASCADED | LOCAL] CHECK OPTION]
    ```
    
### 分区表：只支持水平分区
* 水平分区：指同一表中不同的行记录分配到不同的物理文件中，包括范围划分，HASH划分，轮训划分
* 垂直分区：指同一表中不同的列分配到不同的物理文件中
* 分区类型：
    * 无论哪种分区，如果表中存在主键或者唯一索引，分区列必须属于唯一索引列
        * RANGE分区按列值的连续区间分区
            * 查询分区数据时，查询优化器只能选择简单的内置函数
            * 处理NULL值会将其放入最左边的分区
        * LIST分区按列值的离散区间分区
            * 插入多行数据出错时，MyISAM存储引擎会将前面的插入，出错及以后的数据丢弃；Innodb会当做事务处理，所有数据都会被丢弃
            * 处理NULL值需要在LIST中明确指定NULL存放在哪个分区
        * HASH分区根据用户自定义的HASH值分区
            * 处理NULL值将其转为0存储
        * KEY分区根据mysql自定义的HASH函数分区
            * 处理NULL值将其转为0存储
        * COLUNMS分区（支持非整形数据分区，其他四种只支持整形数据分区）
            * RANGE COLUNMS：针对多个列值进行分区
            * 支持分区的类型包括INT, SMALLINT, TINYINT, BIGINT, DATE, DATETIME, CHAR, VARCHAR, BINARY, VARBINARY
    * 子分区：
        * RANGE和LIST分区允许再使用HASH和KEY做进一步子分区，且每个子分区数量必须相同，并且一旦创立子分区，所有子分区都要建立
        * 子分区的名字必须唯一
* 需不需要创建分区表看数据量的大小，如果分区能够降低B+树的层次，则可以尝试，否则没必要，因此大部分数据例如1000万条也最多3层B+树，因此查询只需要2-3次IO，而分成10区后则需要20次IO
* 创建分区表：
    ```sql
    CREATE [TEMPORARY] TABLE [IF NOT EXISTS] tbl_name
        (create_definition,...)
        [table_options]
        [partition_options]
    CREATE TABLE new_tbl LIKE orig_tbl;
    CREATE TABLE IF NOT EXISTS user_info ( 
        uuid BIGINT NOT NULL PRIMARY KEY
    ) ENGINE= InnoDB 
      PARTITION BY RANGE (uuid) (
          PARTITION p0 VALUES LESS THAN (10),
          PARTITION p1 VALUES LESS THAN (20));
    ```

### 恢复和备份
* 备份类别：
    * 备份方法：
        * 热备份：在数据库运行时直接备份，对正在运行的数据库没有影响，工具：ibbackup, xtrabackup
        * 冷备份：在数据库停止运行时备份，一般就是拷贝数据库文件即可
        * 温备份：在数据库运行时备份，备份时会对数据库加全局锁，对正在运行的数据库造成影响
    * 备份后文件的内容：
        * 逻辑备份：记录的是SQL语句，用户数据库升级、迁移等，非常耗时，工具：mysqldump
        * 裸文件备份：其实就是数据库文件
    * 备份内容：
        * 完全备份：对数据库做一个全面的备份
        * 增量备份：在上次备份的基础上备份
        * 日志备份：利用binlog实现备份

### 字符集编码规则：
* 创建数据库时，将根据服务器上面的character_set_server设定数据库的默认字符集
* 创建表时，根据数据库的字符集创建默认字符集
* 创建列时，根据表的字符集创建默认字符集
* 建立连接时，服务端会认为客户端使用的是character_set_client指定的字符集
* 通信时，服务端接收到客户端的SQL后，会转为character_set_connection指定的字符集
* 返回结果时，服务端会按character_set_result指定的字符集返回给客户端


****

# mysql进阶

### 分布式Mysql集群实战
* https://dev.mysql.com/doc/refman/8.0/en/mysql-innodb-cluster-userguide.html
* https://dev.mysql.com/doc/refman/8.0/en/mysql-innodb-cluster-working-with-cluster.html

### 性能优化
* https://dev.mysql.com/doc/refman/8.0/en/optimization.html

### Mysql Shell
* https://dev.mysql.com/doc/mysql-shell/8.0/en/mysql-shell-features.html

****

# mysql系统参数
* 配置文件：
    * `mysql --help | grep my.cnf` 查看配置文件读取顺序，后面的会覆盖前面的，加载顺序：`my.cnf, MYSQL_TCP_PORT, /etc/my.cnf /etc/mysql/my.cnf /usr/local/mysql/etc/my.cnf ~/.my.cnf`
* `datadir`：指定数据库文件路径
* `autocommit`：设置自动提交
    * 1开启自动提交（隐式提交），系统默认值
    * 0关闭自动提交
* `sort_rows`：指定默认排序的列
* `sql_mode`：指定sql的模式
* `foreign_key_checks`：设置是否开启外键约束检查
* `%engine%`：查询默认存储引擎
* information_schema表：存储所有系统相关信息
    * `SELECT * FROM information_schema.XXX;`：指定查询系统信息，与`SHOW [GLOBAL | SESSION] VARIABLES LIKE 'XXX';`功能相同
    * `INFORMATION_SCHEMA Tables`：
        * https://dev.mysql.com/doc/refman/8.0/en/information-schema.html
        * https://dev.mysql.com/doc/refman/8.0/en/innodb-information-schema.html
* mysql.user表：存储所有用户相关的信息
* `completion_type`：设置COMMIT WORK提交后的行为
    * 0表示没有任何操作，默认值
    * 1表示提交后，马上开启一个相同隔离级别的事务
    * 2表示提交后，直接断开与服务器的连接
* transaction-isolation：设置事务隔离级别
* [@@GLOBAL.|@@]tx_isolation：查看当前事务的隔离级别
* character_set_database：设置数据库默认字符编码
* query_cache_min_res_unit：缓存分配内存块时的最小单位
* query_cache_type：是否打开查询缓存
* query_cache_size：查询缓存使用的总内存大小
* query_cache_limit：缓存最大的结果集，查询结果超过这个值就不会被缓存了
* query_cache_wlock_invalidate：判断当数据被锁后，是否还能从缓存中读出数据，默认是关闭的
* 基础配置选项：<img src='../images/mysql/mysql基础配置选项.png' width='350px'>

****

# SQL
* 语法说明文档：https://dev.mysql.com/doc/refman/8.0/en/sql-syntax-data-definition.html
* ```EXPLAIN SQL```：查看SQL执行计划，解释查询语句，调试SQL
* ```EXPLAIN PARTITIONS SQL```：查询分区搜索
* ```SHOW [GLOBAL | SESSION] VARIABLES LIKE 'XXX';```：查看系统参数
* ```SHOW SLAVE STATUS;```：查询从服务器状态
* ```SHOW MASTER STATUS;```：查询主服务器状态
* ```SHOW TABLE STATUS LIKE 'table_name';```：查询表的状态
* ```SHOW ENGINE innodb STATUS;```：查询存储引擎状态
* ```SHOW CREATE TABLE;```：查询建表的语法解析
* ```SET PROFILING=1; SHOW PROFILES FOR QUERY [N]```：查询剖析工具
* ```SHOW GLOBAL STATUS;```：查看全局服务器配置信息
* ```SHOW STATUS;```：查看会话级别服务器配置信息
* SET和SELECT：设置和查询全局变量
    * SET [GLOBAL | SESSION] system_var_name=expr
    * SET [@@GLOBAL. | @@SESSION. | @@]system_var_name=expr
    * SELECT [GLOBAL | SESSION] system_var_name
    * SELECT [@@GLOBAL. | @@SESSION. | @@]system_var_name
* ```SELECT * FROM table FORCE INDEX(index_name)```：强行使用某个索引查询
* ```SELECT MAX(auto_inc_col_name) FROM table_name FOR UPDATE;```：获取表的下一个自增长id值

* 授权操作：
    ```sql
    GRANT priv_type [(column_list)] [, priv_type [(column_list)]] ...
        ON [object_type] priv_level
        TO user [auth_option] [, user [auth_option]] ...
        [REQUIRE {NONE | tls_option [[AND] tls_option] ...}]
        [WITH {GRANT OPTION | resource_option} ...]
    GRANT ALL ON db1.* TO 'jeffrey'@'localhost';
    GRANT SELECT ON db2.invoice TO 'jeffrey'@'localhost';
    ```
* 取消授权：
    ```sql 
    REVOKE priv_type [(column_list)] [, priv_type [(column_list)]] ...
        ON [object_type] priv_level
        FROM user [, user]
    REVOKE INSERT ON *.* FROM 'jeffrey'@'localhost';
    ```
* 创建用户：
    ```sql
    CREATE USER [IF NOT EXISTS]
        user [auth_option] [, user [auth_option]] ...
        [REQUIRE {NONE | tls_option [[AND] tls_option] ...}]
        [WITH resource_option [resource_option] ...]
        [password_option | lock_option]
    CREATE USER 'jeffrey'@'localhost'
        IDENTIFIED WITH sha256_password BY 'new_password'
        PASSWORD EXPIRE INTERVAL 180 DAY;
    ```
* 创建函数和存储过程：
    ```sql
    CREATE [DEFINER = { user | CURRENT_USER }]
        PROCEDURE sp_name ([proc_parameter[,...]])
        [characteristic ...] routine_body

    CREATE [DEFINER = { user | CURRENT_USER }]
        FUNCTION sp_name ([func_parameter[,...]])
        RETURNS type
        [characteristic ...] routine_body
    ```
* 创建触发器：只支持FOR EACH ROW模式
    ```sql
    CREATE [DEFINER = { user | CURRENT_USER }]
        TRIGGER trigger_name
        trigger_time trigger_event
        ON tbl_name FOR EACH ROW
        [trigger_order]
        trigger_body
    ```

****

# 基准测试
* 基准测试工具：纯测数据库sysbench, mysql-tpcc, mysqlslap, sql-bench，集成测试ab, http_load；实例参考《高性能mysql》第90页
    * sysbench：测试cpu性能，磁盘io性能，调度程序性能，内存分配以及传输速度，POSIX线程性能，数据库OLTP基准测试
    * msql-tpcc：测试数据库OLTP性能，度量单位tmpC(transaction per minute)，越大性能越好
    * ab：Apache的测试HTTP服务器每秒最多可以处理多少请求
    * http_load：与ab类似
    * JMeter：即可以测数据库，也可以集成测试
* 基准测试性能指标：Percona Tooltik工具集监控收集性能相关数据
    * 吞吐量：单位时间内处理的事务量
    * 响应时间或延迟：一次事务处理的时间
    * 并发性：同时可接受响应请求的数量，并发性提高要关注吞吐量是否下降，响应时间是否变长
    * 可扩展性：增加一倍的系统资源，上述三个指标随之增加一倍（理想情况）

****

# 索引
* 创建索引：
    ```sql
    CREATE [UNIQUE|FULLTEXT|SPATIAL] INDEX index_name
        [index_type]
        ON tbl_name (index_col_name,...)
        [index_option]
        [algorithm_option | lock_option]
    ```
* B+树索引的匹配规则：
    * 全值匹配：和索引中的所有列进行匹配，顺序无关，mysql查询优化器会优化谓词顺序已匹配索引列
    * 匹配最左前缀：
        * 在联合索引中，即使谓词只使用了索引的第一列，还是会使用索引查询，与顺序有关
        * 从索引列的最左边开始进行匹配，一旦出现范围查询，范围查询以后的列都不会进行索引匹配
        * 如果谓词省略了中间某些列，也只会使用前面已经匹配的列，例如索引列为A,B,C，谓词为A,C，则索引只会应用到A这一列
    * 匹配列前缀：只匹配某一列的值的开头部分，例如谓词Like 'xxx%'，会走索引，但是会当纳入范围匹配，也就是后面的列都不会走索引了
    * 匹配范围值：
        * 范围列可以用到索引（必须是最左前缀），但是范围列后面的列无法用到索引
        * 注意多值匹配不属于范围值匹配，有些Between操作并不是范围值匹配，而是多值匹配
    * 精确匹配某一列并范围匹配另外一列：使用索引的第一列全匹配，第二列范围匹配
    * 只访问索引的查询：查询结果只需要访问索引，无误访问数据行
    * 索引不匹配的情况：查询条件中含有函数或表达式
* HASH索引的匹配规则：
    * 只能匹配等值操作，并且不支持部分匹配，如果HASH索引计算的是A,B两列的HASH值，则条件谓词中必须且只能包含这两列
    * 自适应HASH：当某一索引值被频繁使用时，会建立一个该值的HASH索引，避免反复的遍历B+树
    * 如果是对于大字符串列，可以手动对该列使用HASH函数编码，并把编码值存入到B+树索引，提高查询速率（避免了大字符串的等值比较），mysql支持的HASH函数：```crc32(), MD5(), SHA1()```
* 三星索引：
    * 索引中包含了条件谓词中的列信息
    * 索引中的列顺序与条件谓词中出现的列顺序一致
    * 索引中的列包含查询语句中出现的所有列
    * 发生冲突时，如果要保存索引自动排序功能，保留第二颗星；如果要尽量缩小索引行数据，保留第一颗星
* 使用索引避免排序规则：
    * 索引的列顺序与ORDER BY字句中列出现的顺序一致，且排序方向一样时可以避免排序操作
    * 当有多个表关联查询时，ORDER BY中出现的列必须全部是第一个表中的列
* 索引和锁：如果查找过程中没有走索引查找，则会做全表扫描并将表锁定